In [ ]:
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        1,              #  1
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    try:
        tmp = DX(str(float(obj)))
        obj = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(obj)):
            tmp = DX(str(float(obj[i])))
            obj[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return obj

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm, deg) = ( m/1000, m/100, pi/180)
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton

half = S(1)/2

# ---

# https://colab.research.google.com/github/kassbohm/tm-snippets/blob/master/ipynb/TM_2/4_BB/2_BL/2.4.2.G-FEM_cc.ipynb

pprint("\nSolution 1:")
# Using full matrix to find deformations and forces:

a, q, EI = var("a, q, EI")
# length of element:
l = a/4
l2 = l*l
l3 = l*l*l

K = EI/l3
K *= Matrix(
[
[  4*l2 ,  -6*l ,  2*l2 ,   6*l ,  0    ,   0    ],
[ -6*l  ,  12   , -6*l  , -12   ,  0    ,   0    ],
[  2*l2 ,  -6*l ,  8*l2 ,   0   ,  2*l2 ,   6*l  ],
[  6*l  , -12   ,  0    ,  24   , -6*l  , -12    ],
[  0    ,   0   ,  2*l2 ,  -6*l ,  4*l2 ,   6*l  ],
[  0    ,   0   ,  6*l  , -12   ,  6*l  ,  12    ],
]
)
w1,p2,w2,p3 = var("w1,p2,w2,p3")
M1,F3 = var("M1,F3")
u = Matrix([0,w1,p2,w2,p3,0])
f = Matrix([M1,0,0,0,0,F3]) + q * Matrix([-l2/12, l/2, 0, l,l2/12, l/2 ])

unknowns = [w1,p2,w2,p3,M1,F3]

eq = Eq(K*u , f)
sol = solve(eq, unknowns)
w1,p2,w2,p3 = sol[w1],sol[p2],sol[w2],sol[p3]

l, B = var("l, B")
sub_list = [
    (a, 4*l),
    (EI, B),
    ]

pprint("\n(w1, p2, w2, p3) / (l³ q / EI):")
for x in [w1,p2,w2,p3]:
    tmp = x.subs(sub_list)
    tmp /= l**3*q/B
    # pprint(latex(tmp))
    print(tmp)

pprint("\n(w1, ψ2, w2, ψ3) / ( q a³ / (EI) ):")
for x in [w1,p2,w2,p3]:
    tmp = x / (q*a**3 / EI)
    pprint(tmp)

pprint("\nSolution 2:")
# Klein:
sub_list_Klein=[
    (a, 2 *m),
    (q, 1 *Newton/m),
    (EI, 1 *Newton*m**2),
    ]

# Find deformations only:
K = Matrix(
    [
    [    12   ,  -6*l  ,  -12 ,  0      ],
    [  -6*l   ,  8*l2  ,    0 ,  2*l2   ],
    [   -12   ,  0     ,   24 ,  -6*l   ],
    [   0     ,  2*l2  ,  -6*l ,  4*l2  ],
    ])

K *= EI/l3

w1, p2, w2, p3 = var("w1, p2, w2, p3")
u = Matrix([w1, p2, w2, p3])
f = q * Matrix([l/2, 0, l, l2/12])

eq = Eq(K*u , f)
sol = solve(eq, [w1,p2,w2,p3])
w1, p2, w2, p3 = sol[w1], sol[p2], sol[w2], sol[p3]

pprint("\n(w1, ψ2, w2, ψ3) / ( q a³ / (EI) ):")
fac = a**3*q/EI
tmp = w1
tmp = tmp.subs(l, a/4)
pprint(tmp/fac)
tmp = tmp.subs(sub_list_Klein)
tmp /= m
tmp = iso_round(tmp, 0.001)
pprint(tmp)

tmp = p2
tmp = tmp.subs(l, a/4)
pprint(tmp/fac)
tmp = tmp.subs(sub_list_Klein)
tmp = iso_round(tmp, 0.001)
pprint(tmp)

tmp = w2
tmp = tmp.subs(l, a/4)
pprint(tmp/fac)
tmp = tmp.subs(sub_list_Klein)
tmp /= m
tmp = iso_round(tmp, 0.001)
pprint(tmp)

tmp = p3
tmp = tmp.subs(l, a/4)
pprint(tmp/fac)
tmp = tmp.subs(sub_list_Klein)
tmp = iso_round(tmp, 0.001)
pprint(tmp)


# Section Loads as in theory for 2-node-beam:
x, xi, l, B, q = var("x, xi, l, B, q")

N1 = -xi**3 + 2*xi**2 - xi
N2 = 2*xi**3 - 3*xi**2 + 1
N3 = -xi**3 + xi**2
N4 = -2*xi**3 + 3*xi**2
N5 = xi**4/24 - xi**3/12 + xi**2/24

N = Matrix([l*N1, N2, l*N3, N4, l**4/B * N5])
dNx = diff(N, xi) / l
d2Nx = diff(dNx, xi) / l

pprint("\nA:")
A = - B * d2Nx
fac = l**3*q/B
w1 = fac * 10*l/3
p2 = fac * S(11)/6
w2 = fac * 19*l/8
p3 = fac * S(8)/3

pprint("\n- B w'':")
u1 = Matrix([0, w1, p2, w2, q])
u2 = Matrix([p2, w2, p3, 0, q])
tmp = A.T*u1
tmp = tmp[0]
tmp = tmp.simplify()
pprint(tmp)
a = var("a")
tmp = tmp.subs(l, a/4)
pprint(tmp)
